### Goal: to prepare the dataset for exploration and modeling
- data source: a sample of users' records from Riiid database

In [1]:
# General Imports 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Warnings 
import warnings
warnings.filterwarnings("ignore")

# Monitor memory use and time
import psutil
from tqdm import tqdm

# Import the prepare.py
import prepare

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif

### Reading Files

In [2]:
# Reading in data from local csv files
df_train = pd.read_csv('data_2000users/train.csv')
df_validate = pd.read_csv('data_2000users/validate.csv')
df_test = pd.read_csv('data_2000users/test.csv')

# Print how much memory has been used
print("Momery usage: ", psutil.virtual_memory().percent)

# Print the shapes of the loaded files
df_train.shape, df_validate.shape, df_test.shape

Momery usage:  43.5


((411517, 18), (50842, 18), (52868, 18))

In [3]:
# Read the questions_with_tag_counts.csv
df_ques = pd.read_csv('questions_with_tag_counts.csv', index_col=0)
# df_lects = pd.read_csv('lectures_with_part_name.csv', index_col=0)

# Print how much memory has been used
print("Momoery usage: ", psutil.virtual_memory().percent)

# Print its shape
df_ques.shape

Momoery usage:  43.1


(13523, 6)

### Drop columns from Kaggle questions.csv and lectures.csv

In [4]:
# Define the columns need to be dropped
cols = ['lecture_id', 'tag', 'lecture_part', 'type_of', 'question_id',
        'bundle_id', 'correct_answer', 'question_part', 'tags']

# Drop the columns in train, validate, and test dataset
df_train = df_train.drop(columns = cols)
df_validate = df_validate.drop(columns = cols)
df_test = df_test.drop(columns = cols)

# Print how much memory has been used
print("Momoery usage: ", psutil.virtual_memory().percent)

# Inspect the shapes of the dataframes
df_train.shape, df_validate.shape, df_test.shape

Momoery usage:  43.2


((411517, 9), (50842, 9), (52868, 9))

### Add features:
- user_acc_mean
- user_lectures_running_total
- avg_user_q_time

In [5]:
%%time
# Calculate how much time it takes to run the cell

# Use the helper function in prepare.py to add the new features
train = prepare.sam_train_features(df_train)
validate = prepare.sam_valtest_features(train, df_validate)
test = prepare.sam_valtest_features(train, df_test)

# Print how much memory has been used
print("Momoery usage: ", psutil.virtual_memory().percent)

# Print the shapes of the dataframes
train.shape, validate.shape, test.shape

Momoery usage:  43.3
CPU times: user 2.55 s, sys: 198 ms, total: 2.75 s
Wall time: 2.79 s


((411517, 13), (50842, 12), (52868, 12))

In [6]:
# # Drop user ids to save memory

# train.drop(columns='user_id', inplace=True)
# validate.drop(columns='user_id', inplace=True)
# test.drop(columns='user_id', inplace=True)

# train.head(1)

### Handle nulls and the np.inf

In [7]:
# Inspect the train dataset before
train.head(2)

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,user_acc_mean,user_lectures_running_total,q_time,avg_user_q_time
0,0,1864702,5720,0,0,1,1,NaN,NaN,0.630049,0,45951.0,11917302.0
1,45951,1864702,5204,0,1,1,0,inf,False,0.630049,0,28391.0,11917302.0


In [8]:
# Use the helper function in prepare.py to handle the nulls
train = prepare.handle_null(train)
validate = prepare.handle_null(validate)
test = prepare.handle_null(test)
    
# Use the helper function in prepare.py to handle the np.inf
train = prepare.handle_inf(train)
validate = prepare.handle_inf(validate)
test = prepare.handle_inf(test)

# Inspect the train after
train.head(2)

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,user_acc_mean,user_lectures_running_total,q_time,avg_user_q_time
0,0,1864702,5720,0,0,1,1,0.0,False,0.630049,0,45951.0,11917302.0
1,45951,1864702,5204,0,1,1,0,0.0,False,0.630049,0,28391.0,11917302.0


In [9]:
# # Define a function to separate the lecture rows and question rows

# def seperate_rows(df):
#     '''
#     separate the lecture rows and question rows
#     '''
#     mask_question = (df['answered_correctly'] != -1)
#     mask_lecture = (df['answered_correctly'] == -1)
#     df_question = df[mask_question]
#     df_lecture = df[mask_lecture]
#     return df_question, df_lecture

In [10]:
# # Apply the function on train, validate, and test

# train, train_lects = seperate_rows(train)
# validate, validate_lects = seperate_rows(validate)
# test, test_lects = seperate_rows(test)

# train.shape, train_lects.shape

### Drop the lecture rows

In [11]:
# Print the shape of the train before the process
train.shape

(411517, 13)

In [12]:
# Use the helper function in prepare.py to drop the lecture rows in train/validate/test
train = prepare.drop_lecture_rows(train)
validate = prepare.drop_lecture_rows(validate)
test = prepare.drop_lecture_rows(test)

# Print the shape of the train after
train.shape

(403377, 13)

### Merge: 
- train, validate and test with df_ques
- train_lects with df_lects: not done yet

In [13]:
# Merge train/validate/test with df_ques

train = train.merge(df_ques, how='left', left_on='content_id', right_on='question_id')
validate = validate.merge(df_ques, how='left', left_on='content_id', right_on='question_id')
test = test.merge(df_ques, how='left', left_on='content_id', right_on='question_id')

# Print how much memory has been used
print("Momoery usage: ", psutil.virtual_memory().percent)

# Print the shapes of the dataframes
train.shape, validate.shape, test.shape

Momoery usage:  44.1


((403377, 19), (49945, 18), (51971, 18))

In [14]:
# Double check to make sure the merge is successful
train.head(1)

,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,user_acc_mean,user_lectures_running_total,q_time,avg_user_q_time,question_id,bundle_id,correct_answer,part,tags,tag_count
0,0,1864702,5720,0,0,1,1,0.0,False,0.630049,0,45951.0,11917302.0,5720,5720,1,5,115,1


**Columns to drop**
- timestamp: used to compute avg time each user to answer 1 question
- content_id (question_id): after drop the lecture rows, used to compute average question mean
- content_type_id: **drop**
- task_container_id: used to compute avereage task mean
- user_answer: **drop**
- answered_correctly: target varibale
- prior_question_elapsed_time: **drop**
- prior_question_had_explanation: used for question_had_explanation
- correct_answer: **drop**

### Drop the redundant columns to save memory

In [15]:
# Drop the redundant column to save memory

train.drop(columns=['content_type_id', 'user_answer', 
                    'prior_question_elapsed_time', 'correct_answer'], inplace=True)
validate.drop(columns=['content_type_id', 'user_answer', 
                       'prior_question_elapsed_time', 'correct_answer'], inplace=True)
test.drop(columns=['content_type_id', 'user_answer', 
                   'prior_question_elapsed_time', 'correct_answer'], inplace=True)

# Print how much memory has been used
print("Momoery usage: ", psutil.virtual_memory().percent)

# Print the shapes of the dataframes
train.shape, validate.shape, test.shape

Momoery usage:  44.6


((403377, 15), (49945, 14), (51971, 14))

In [16]:
# Double check if the columns above has been removed
train.head(1)

,timestamp,user_id,content_id,task_container_id,answered_correctly,prior_question_had_explanation,user_acc_mean,user_lectures_running_total,q_time,avg_user_q_time,question_id,bundle_id,part,tags,tag_count
0,0,1864702,5720,0,1,False,0.630049,0,45951.0,11917302.0,5720,5720,5,115,1


### Add features:
- part accuracy
- bundle accuray

In [17]:
train.shape

(403377, 15)

In [18]:
def part_bundle_features(train, validate, test):
    
    # Calculate the average accuracy for each unique bundle id
    bundle_accuracy = train.groupby(['bundle_id'])['answered_correctly'].mean().round(2).to_frame().reset_index()
    bundle_accuracy.columns = ['bundle_id', 'mean_bundle_accuracy']
    
    # Add bundle mean accuracy as a feature to train, validate, and test
    merged_train = train.merge(bundle_accuracy, left_on='bundle_id', right_on='bundle_id', how='left')
    merged_validate = validate.merge(bundle_accuracy, left_on='bundle_id', right_on='bundle_id', how='left')
    merged_test = test.merge(bundle_accuracy, left_on='bundle_id', right_on='bundle_id', how='left')
    
    # Calculate the average part accuracy
    tag_accuracy = train.groupby(['part'])['answered_correctly'].agg(['mean']).round(2).reset_index()
    tag_accuracy.columns = ['part', 'mean_part_accuracy']
    
    # Add average part accuracy
    train_df = merged_train.merge(tag_accuracy, left_on='part', right_on='part')
    validate_df = merged_validate.merge(tag_accuracy, left_on='part', right_on='part')
    test_df = merged_test.merge(tag_accuracy, left_on='part', right_on='part')
    
    # Calculate the mean container accuracy for each part
    tag_bundles = train.groupby(['question_id', 'task_container_id', 'part'])['answered_correctly'].mean().round(2).reset_index()
    tag_bundles.rename(columns={'answered_correctly': 'mean_container_part_accuracy'}, inplace=True)
#     tag_bundles.drop(columns='question_id', inplace=True)
    
    # Add mean container part accuracy
    train_set = train_df.merge(tag_bundles, how='left', 
                               left_on=['task_container_id', 'part', 'question_id'], 
                               right_on=['task_container_id', 'part', 'question_id'])
    
    validate_set = validate_df.merge(tag_bundles, how='left', 
                                     left_on=['task_container_id', 'part', 'question_id'], 
                                     right_on=['task_container_id', 'part', 'question_id'])
    
    test_set = test_df.merge(tag_bundles, how='left', 
                             left_on=['task_container_id', 'part', 'question_id'], 
                             right_on=['task_container_id', 'part', 'question_id'])

    
    return train_set, validate_set, test_set

In [19]:
%%time
# Compute the time to process the cell

# Use the function defined above to add the new features
train, validate, test = part_bundle_features(train, validate, test)

# Print how much memory has been used
print("Momoery usage: ", psutil.virtual_memory().percent)

# Print the shapes of the dataframes
train.shape, validate.shape, test.shape

Momoery usage:  48.1
CPU times: user 598 ms, sys: 150 ms, total: 748 ms
Wall time: 762 ms


((403377, 18), (49945, 17), (51971, 17))

In [20]:
# Make sure the new features have been added
train.head(1)

,timestamp,user_id,content_id,task_container_id,answered_correctly,prior_question_had_explanation,user_acc_mean,user_lectures_running_total,q_time,avg_user_q_time,question_id,bundle_id,part,tags,tag_count,mean_bundle_accuracy,mean_part_accuracy,mean_container_part_accuracy
0,0,1864702,5720,0,1,False,0.630049,0,45951.0,11917302.0,5720,5720,5,115,1,0.82,0.61,1.0


### Add features
- content accuracy
- task accuracy

In [22]:
%%time
# Compute the time to process the cell

# Use the helper function in prepare.py to add the features
train = prepare.merge_with_stats_train(train)
validate = prepare.merge_with_stats_valortest(train, validate)
test = prepare.merge_with_stats_valortest(train, test)

# Print how much memory has been used
print("Momoery usage: ", psutil.virtual_memory().percent)

# Print the shapes of the dataframes
train.shape, validate.shape, test.shape

Momoery usage:  60.5
CPU times: user 6.7 s, sys: 137 ms, total: 6.83 s
Wall time: 6.88 s


((403377, 19), (49945, 18), (51971, 18))

In [23]:
# Check the result
train.head(1)

,timestamp,user_id,content_id,task_container_id,answered_correctly,prior_question_had_explanation,user_acc_mean,user_lectures_running_total,q_time,avg_user_q_time,question_id,bundle_id,part,tags,tag_count,mean_bundle_accuracy,mean_part_accuracy,mean_content_accuracy,mean_task_accuracy
0,0,1864702,5720,0,1,False,0.630049,0,45951.0,11917302.0,5720,5720,5,115,1,0.82,0.61,0.818182,0.682248


### Add features:
- mean tagcount accuracy
- mean tags accuracy

In [32]:
# Define the functions to add the features to train/validate/test

def mean_tagcount_accuracy(df):
    tagcount_accuracy = df.groupby('tag_count').answered_correctly.mean().round(2).rename('mean_tagcount_accuracy')
    return tagcount_accuracy

def mean_tag_accuracy(df):
    tags_accuracy = df.groupby('tags').answered_correctly.mean().round(2).rename('mean_tags_accuracy')
    return tags_accuracy

def tag_features(train, validate, test):
    
    tagcount_accuracy = mean_tagcount_accuracy(train)
    tags_accuracy = mean_tag_accuracy(train)

    train = train.merge(tagcount_accuracy, how='left', on='tag_count')
    train = train.merge(tags_accuracy, how='left', on='tags')

    validate = validate.merge(tagcount_accuracy, how='left', on='tag_count')
    validate = validate.merge(tags_accuracy, how='left', on='tags')

    test = test.merge(tagcount_accuracy, how='left', on='tag_count')
    test = test.merge(tags_accuracy, how='left', on='tags')

    return train, validate, test

In [35]:
# Used the defined function to add the features
train, validate, test = tag_features(train, validate, test)

# Check the result
train.head(1)

### Fill the Nulls

In [37]:
# Compute the missing values in the validate dataset before filling the nulls
validate.isnull().sum(axis=0)

timestamp                            0
user_id                              0
content_id                           0
task_container_id                    0
answered_correctly                   0
prior_question_had_explanation       0
user_acc_mean                        0
user_lectures_running_total          0
avg_user_q_time                      0
question_id                          0
bundle_id                            0
part                                 0
tags                                 0
tag_count                            0
mean_bundle_accuracy               206
mean_part_accuracy                   0
mean_content_accuracy              206
mean_task_accuracy                1405
mean_tagcount_accuracy               0
mean_tags_accuracy                   4
dtype: int64

In [38]:
# Use the helper function in prepare.py to fill the nulls
validate = prepare.fill_nulls(validate)
test = prepare.fill_nulls(test)

# Check the result
validate.isnull().sum(axis=0)

timestamp                         0
user_id                           0
content_id                        0
task_container_id                 0
answered_correctly                0
prior_question_had_explanation    0
user_acc_mean                     0
user_lectures_running_total       0
avg_user_q_time                   0
question_id                       0
bundle_id                         0
part                              0
tags                              0
tag_count                         0
mean_bundle_accuracy              0
mean_part_accuracy                0
mean_content_accuracy             0
mean_task_accuracy                0
mean_tagcount_accuracy            0
mean_tags_accuracy                0
dtype: int64

### Shift prior question had explanation to current question

In [43]:
# shift prior question had explanation to current question

train.prior_question_had_explanation = train.prior_question_had_explanation.shift(-1)
validate.prior_question_had_explanation = validate.prior_question_had_explanation.shift(-1)
test.prior_question_had_explanation = test.prior_question_had_explanation.shift(-1)

train = train.rename(columns={"prior_question_had_explanation": "question_had_explanation"})
validate = validate.rename(columns={"prior_question_had_explanation": "question_had_explanation"})
test = test.rename(columns={"prior_question_had_explanation": "question_had_explanation"})

In [45]:
# train.to_csv('train_exploration.csv')
# validate.to_csv('validate_exploration.csv')
# test.to_csv('test_exploration.csv')

### Drop the q_time in the train

In [47]:
# Drop column q_time in the train
train_s = train.drop(columns='q_time')
train_s.head(1)

,timestamp,user_id,content_id,task_container_id,answered_correctly,question_had_explanation,user_acc_mean,user_lectures_running_total,avg_user_q_time,question_id,bundle_id,part,tags,tag_count,mean_bundle_accuracy,mean_part_accuracy,mean_content_accuracy,mean_task_accuracy,mean_tagcount_accuracy,mean_tags_accuracy
0,0,1864702,5720,0,1,False,0.630049,0,11917302.0,5720,5720,5,115,1,0.82,0.61,0.818182,0.682248,0.62,0.79


### Drop columns not needed for modeling

In [48]:
# Define the function to drop the unnecessary columns for modeling

def drop_columns(df):
    """
    Accepts df and drops various columns that are not needed for modeling.
    """
    cols = ['timestamp', 'user_id', 'content_id', 'task_container_id',
            'question_id', 'bundle_id', 'part', 'tags', 'tag_count']
    df.drop(columns=cols, inplace=True)
    return df

In [49]:
# Print the shapes of the dataframes before dropping
train_s.shape, validate.shape, test.shape

((403377, 20), (49945, 20), (51971, 20))

In [50]:
# Use the defined function to drop the columns
train_s = drop_columns(train_s)
validate_s = drop_columns(validate)
test_s = drop_columns(test)

# Check the result
train_s.shape, validate_s.shape, test_s.shape

((403377, 11), (49945, 11), (51971, 11))

### Convert boolean to num

In [51]:
# Define the function to convert the boolean values to the numeric

def boolean_to_num(df):
    """
    Accepts df. Converts True and False values into 1's and 0's resepectively, within the 
    question_had_explanation column.
    """
    df = df.fillna(False)
    m = df.question_had_explanation.apply(lambda i: 1 if i == True else 0)
    df.question_had_explanation = m
    return df

In [52]:
# Use the defined function to do the conversion

train_s = boolean_to_num(train_s)
validate_s = boolean_to_num(validate_s)
test_s = boolean_to_num(test_s)

In [54]:
# Check the result
test_s.dtypes

answered_correctly               int64
question_had_explanation         int64
user_acc_mean                  float64
user_lectures_running_total      int64
avg_user_q_time                float64
mean_bundle_accuracy           float64
mean_part_accuracy             float64
mean_content_accuracy          float64
mean_task_accuracy             float64
mean_tagcount_accuracy         float64
mean_tags_accuracy             float64
dtype: object

### Scale before modeling

In [55]:
# Define the fucntion to scale the datasets

def scale(train, validate, test, columns_to_scale):
    '''
    Accepts train, validate, test and list of columns to scale. Scales listed columns.
    '''
    new_column_names = [c + '_scaled' for c in columns_to_scale]
    
    scaler = MinMaxScaler()
    scaler = scaler.fit(train[columns_to_scale])

    train = pd.concat([
        train,
        pd.DataFrame(scaler.transform(train[columns_to_scale]), columns=new_column_names, index=train.index),
    ], axis=1)

    validate = pd.concat([
        validate,
        pd.DataFrame(scaler.transform(validate[columns_to_scale]), columns=new_column_names, index=validate.index),
    ], axis=1)

    test = pd.concat([
        test,
        pd.DataFrame(scaler.transform(test[columns_to_scale]), columns=new_column_names, index=test.index),
    ], axis=1)
    
    train.drop(columns=columns_to_scale, inplace=True)
    validate.drop(columns=columns_to_scale, inplace=True)
    test.drop(columns=columns_to_scale, inplace=True)
    
    return train, validate, test

In [56]:
# Define the columns needs to be scaled
columns_to_scale = ['user_lectures_running_total', 'avg_user_q_time']

# Use the defined function to scaled the datasets
train_s, validate_s, test_s = scale(train_s, validate_s, test_s, columns_to_scale)

In [57]:
# Check results
train_s.head(1)

,answered_correctly,question_had_explanation,user_acc_mean,mean_bundle_accuracy,mean_part_accuracy,mean_content_accuracy,mean_task_accuracy,mean_tagcount_accuracy,mean_tags_accuracy,user_lectures_running_total_scaled,avg_user_q_time_scaled
0,1,0,0.630049,0.82,0.61,0.818182,0.682248,0.62,0.79,0.0,0.001202


### Create one function to complete all the preparation

In [4]:
%%time
# Compute the time to process the cell

# Test the fucntion to complete all the steps above
train, validate, test, train_s, validate_s, test_s = prepare.prep_riiid(df_train, df_validate, df_test)

CPU times: user 9.71 s, sys: 725 ms, total: 10.4 s
Wall time: 10.4 s


In [5]:
# Print the shapes of the train/validate/test for exploratioin
train.shape, validate.shape, test.shape

((403377, 21), (49945, 20), (51971, 20))

In [7]:
# Inspect the columns of the train dataset
train.head(1)

,timestamp,user_id,content_id,task_container_id,answered_correctly,question_had_explanation,user_acc_mean,user_lectures_running_total,q_time,avg_user_q_time,...,bundle_id,part,tags,tag_count,mean_bundle_accuracy,mean_part_accuracy,mean_content_accuracy,mean_task_accuracy,mean_tagcount_accuracy,mean_tags_accuracy
0,0,1864702,5720,0,1,False,0.630049,0,45951.0,11917302.0,...,5720,5,115,1,0.82,0.61,0.82,0.68,0.62,0.79


In [6]:
# Print the shapes of the scaled train/validate/test for modeling
train_s.shape, validate_s.shape, test_s.shape

((403377, 11), (49945, 11), (51971, 11))

In [8]:
# Inspect the columns of the scaled train dataset
train_s.head(1)

,answered_correctly,question_had_explanation,user_acc_mean,mean_bundle_accuracy,mean_part_accuracy,mean_content_accuracy,mean_task_accuracy,mean_tagcount_accuracy,mean_tags_accuracy,user_lectures_running_total_scaled,avg_user_q_time_scaled
0,1,0,0.630049,0.82,0.61,0.82,0.68,0.62,0.79,0.0,0.001202


### Select K Best

In [58]:
# Creating train, validate, test DFs that only include non-target variables
X_train = train_s.drop(columns='answered_correctly')
y_train = train_s['answered_correctly']

X_validate = validate_s.drop(columns='answered_correctly')
y_validate = validate_s['answered_correctly']

X_test = test_s.drop(columns='answered_correctly')
y_test = test_s['answered_correctly']

In [60]:
# Define the function to rank all the features

def KBest_ranker(X, y, n):
    '''
    Returns the top n selected features with their scores based on the SelectKBest calss
    Parameters: scaled predictors(X) in df, target(y) in df, the number of features to select(n)
    '''

    # parameters: f_regression stats test, give me 5 features
    f_selector = SelectKBest(f_classif, k=n)

    # Fit on X and y
    f_selector.fit(X, y)

    # Calculate the score for each feature 
    feature_score = f_selector.scores_.round(2)

    # Put the features and their score in a dataframe
    df_features = pd.DataFrame({'features': X.columns, 
                                'score': feature_score})

    # Sort the features based on their score
    df_features.sort_values(by="score", ascending=False, inplace=True, ignore_index=True)

    # Compute how many features in X
    m = X.shape[1]
    
    # Add a rank column
    df_features['rank'] = range(1, m+1)
    
    return df_features[:n]

In [61]:
# Use the defined function to rank all the features in the scaled train dataset
df = KBest_ranker(X_train, y_train, 10)
df

,features,score,rank
0,mean_content_accuracy,76690.75,1
1,mean_bundle_accuracy,55971.00,2
2,user_acc_mean,22419.10,3
3,mean_tags_accuracy,21599.32,4
4,mean_task_accuracy,15910.84,5
5,question_had_explanation,3954.75,6
6,mean_part_accuracy,3397.75,7
7,mean_tagcount_accuracy,2389.41,8
8,user_lectures_running_total_scaled,211.82,9
9,avg_user_q_time_scaled,116.95,10
